# Weight_Regularization
torch.optim.SGD(params, lr=1, momentum=0, dampening=0, weight_decay=0, nesterov=False)

In [1]:
# 모듈 임포트
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [11]:
# 하이퍼파라미터 설정
batch_size = 16
learning_rate = 0.0002
num_epoch = 1

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
# 데이터 로드
mnist_train = dsets.MNIST(root='./', train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root='./', train=False, transform=transforms.ToTensor(), download=True)

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 9.91M/9.91M [00:02<00:00, 4.59MB/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 28.9k/28.9k [00:00<00:00, 58.6kB/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.27MB/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 4.54k/4.54k [00:00<00:00, 10.9MB/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [9]:
# 데이터셋 체크
print(mnist_train[0][0].size(), len(mnist_train))
print(mnist_test[0][0].size(), len(mnist_test))

torch.Size([1, 28, 28]) 60000
torch.Size([1, 28, 28]) 10000


In [10]:
# 데이터로더 설정
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

In [12]:
# CNN 모델
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer = nn.Sequential(
        nn.Conv2d(1, 16, 3, padding=1), # 28
        nn.ReLU(),
        nn.Conv2d(16, 32, 3, padding=1), # 28
        nn.ReLU(),
        nn.MaxPool2d(2, 2), # 14
        nn.Conv2d(32, 64, 3, padding=1), # 14
        nn.ReLU(),
        nn.MaxPool2d(2, 2) # 7
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(64 * 7 * 7, 100),
        nn.ReLU(),
        nn.Linear(100, 10)
    )

  def forward(self, x):
    out = self.layer(x)
    out = out.view(out.size(0), -1)
    out = self.fc_layer(out)

    return out

model = CNN().to(device)

In [13]:
# 손실 함수 및 옵티마이저
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.1) # L2 규제 적용

In [14]:
# Train
total_batch = len(train_loader)
for epoch in range(num_epoch):
  avg_cost = 0
  for X, Y in train_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    output = model(X)
    cost = criterion(output, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print('[Epoch {}] Cost : {}'.format(epoch + 1, avg_cost))

[Epoch 1] Cost : 2.2979650497436523


In [22]:
# 테스트
with torch.no_grad():
  X = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y = mnist_test.targets.to(device)

  output = model(X)
  prediction = torch.argmax(output, dim=1) == Y
  correct_prediction = prediction.float().sum()

print('정확도 : {}'.format(100 * correct_prediction / len(mnist_test)))

정확도 : 18.68000030517578


# Dropout
nn.Dropout(p=0.5, inplace=False)

In [29]:
# 모듈 임포트
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [30]:
# 하이퍼파라미터 설정
batch_size = 16
learning_rate = 0.0002
num_epoch = 1

In [31]:
# 데이터
mnist_train = dsets.MNIST('./', train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST('./', train=False, transform=transforms.ToTensor(), download=True)

In [32]:
# 데이터로더 설정
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)

In [34]:
# 모델 설정
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer = nn.Sequential(
        nn.Conv2d(1, 16, 3, padding=1),
        nn.ReLU(),
        # nn.Dropout2d(0.2), 0.2는 드롭아웃 확률
        nn.Conv2d(16, 32, 3, padding=1),
        nn.ReLU(),
        # nn.Dropout2d(0.2),
        nn.MaxPool2d(2, 2),
        nn.Conv2d(32, 64, 3, padding=1),
        nn.ReLU(),
        # nn.Dropout2d(0.2),
        nn.MaxPool2d(2, 2)
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(64 * 7 * 7, 100),
        nn.ReLU(),
        nn.Linear(100, 10)
    )

  def forward(self, x):
    out = self.layer(x)
    out = out.view(out.size(0), -1)
    out = self.fc_layer(out)
    return out

model = CNN().to(device)

In [35]:
# 손실 함수 및 옵티마이저
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [36]:
# Train
total_batch = len(train_loader)
for epoch in range(num_epoch):
  avg_cost = 0
  for X, Y in train_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    output = model(X)
    cost = criterion(output, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print('Epoch : {}, Cost : {}'.format(epoch + 1, avg_cost))

Epoch : 1, Cost : 2.2964842319488525


In [50]:
# 테스트
model.eval() # 배치 정규화, 드롭아웃 등을 비활성화

with torch.no_grad():
  X = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y = mnist_test.targets.to(device)

  output = model(X)
  prediction = torch.argmax(output, dim=1) == Y
  correct_prediction = prediction.float().sum()

print('정확도 : {}'.format(100 * correct_prediction / len(mnist_test)))

정확도 : 25.079999923706055


# Data_Augmentation
- Resize
- Crop
- Flip
- Rotate (using PIL Transform)

In [51]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

In [52]:
# 하이퍼파라미터 설정
batch_size = 16
learning_rate = 0.0002
num_epoch = 2

In [54]:
# 데이터 다운로드
mnist_train = dsets.MNIST('./', train=True,
                          transform = transforms.Compose([
                              transforms.Resize((34, 34)),
                              transforms.CenterCrop(28),
                              transforms.RandomHorizontalFlip(),
                              transforms.ToTensor(),
                          ]),
                          download=True)
mnist_test = dsets.MNIST('./', train=False, transform=transforms.ToTensor(), download=True)

In [55]:
# 데이터셋 확인
print(mnist_train[0][0].size(), len(mnist_train))
print(mnist_test[0][0].size(), len(mnist_test))

torch.Size([1, 28, 28]) 60000
torch.Size([1, 28, 28]) 10000


In [56]:
# 데이터로더
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)

In [57]:
# CNN 모델
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer = nn.Sequential(
        nn.Conv2d(1, 16, 3, padding=1),
        nn.ReLU(),
        nn.Conv2d(16,  32, 3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Conv2d(32, 64, 3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(64 * 7 * 7, 100),
        nn.ReLU(),
        nn.Linear(100, 10)
    )

  def forward(self, x):
    out = self.layer(x)
    out = out.view(out.size(0), -1)
    out = self.fc_layer(out)
    return out

model = CNN().to(device)

In [58]:
# 손실함수 및 옵티마이저
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [59]:
# 학습
total_batch = len(train_loader)
for epoch in range(num_epoch):
  avg_cost = 0
  for X, Y in train_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    output = model(X)
    cost = criterion(output, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print('Epoch : {}, Cost : {}'.format(epoch + 1, avg_cost))

Epoch : 1, Cost : 2.302511692047119
Epoch : 2, Cost : 2.298917055130005


In [60]:
total_batch

3750

In [62]:
# 테스트
with torch.no_grad():
  X = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y = mnist_test.targets.to(device)

  output = model(X)
  prediction = torch.argmax(output, dim=1) == Y
  correct_prediction = prediction.float().sum()

print('정확도 : {}'.format(100 * correct_prediction / len(mnist_test)))


정확도 : 26.349998474121094


# Weight Initialization
- Initialize with small numbers
- Xavier Initialization
- Kaiming He Initialization

In [63]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [64]:
# 하이퍼파라미터 설정
batch_size = 16
learning_rate = 0.0002
num_epoch = 1

In [66]:
mnist_train = dsets.MNIST("./", train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST("./", train=False, transform=transforms.ToTensor(), download=True)

In [67]:
# 데이터로더 설정
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)

In [78]:
# CNN 모델
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer = nn.Sequential(
        nn.Conv2d(1, 16, 3, padding=1),
        nn.ReLU(),
        nn.Conv2d(16, 32, 3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Conv2d(32, 64, 3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(64 * 7 * 7, 100),
        nn.ReLU(),
        nn.Linear(100, 10)
    )

    # initialization

    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        '''# Init with small numbers
        m.weight.data.normal_(0.0, 0.02)
        m.bias.data.fill_(0)

        # Xavier Initialization
        init.xavier_normal(m.weight.data)
        m.bias.data.fill_(0)'''

        # Kaming Initialization
        init.kaiming_normal_(m.weight)
        m.bias.data.fill_(0)

      elif isinstance(m, nn.Linear):
        init.kaiming_normal_(m.weight)
        m.bias.data.fill_(0)

  def forward(self, x):
    out = self.layer(x)
    out = out.view(out.size(0), -1)
    out = self.fc_layer(out)

    return out

model = CNN().to(device)

In [76]:
# 손실함수 및 옵티마이저
loss_func = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [79]:
# 학습
total_batch = len(train_loader)
for epoch in range(num_epoch):
  avg_cost = 0
  for X, Y in train_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    output = model(X)
    cost = loss_func(output, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print('Epoch : {}, Cost : {}'.format(epoch + 1, avg_cost))

Epoch : 1, Cost : 3.297081708908081


In [83]:
# 학습

with torch.no_grad():
  X = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y = mnist_test.targets.to(device)

  output = model(X)
  prediction = torch.argmax(output, dim=1) == Y
  correct_prediction = prediction.float().sum()

print('정확도 : {}'.format(100 * correct_prediction / len(mnist_test)))

정확도 : 15.289999961853027


# Learning Rate Scheduler
- optim.lr_scheduler.StepLR()
- optim.lr_scheduler.MultiStepLR()
- optim.lr_scheduler.ExponentialLR()
- optim.lr_scheduler.ReduceLROnPlateau()

In [84]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader

In [85]:
# 하이퍼 파라미터
batch_size = 16
learning_rate = 0.001
num_epoch = 2

In [86]:
# 데이터
mnist_train = dsets.MNIST('./', train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST('./', train=False, transform=transforms.ToTensor(), download=True)

In [87]:
# CNN 모델
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer = nn.Sequential(
        nn.Conv2d(1, 16, 3, padding=1),
        nn.ReLU(),
        nn.Conv2d(16, 32, 3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Conv2d(32, 64, 3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(64*7*7, 100),
        nn.ReLU(),
        nn.Linear(100, 10)
    )

  def forward(self, x):
    out = self.layer(x)
    out = out.view(out.size(0), -1)
    out = self.fc_layer(out)

    return out

model = CNN().to(device)

In [89]:
# 손실 함수 및 옵티마이저
loss_func = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.99)

print(dir(scheduler))

['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_get_closed_form_lr', '_get_lr_called_within_step', '_initial_step', '_last_lr', '_step_count', 'base_lrs', 'gamma', 'get_last_lr', 'get_lr', 'last_epoch', 'load_state_dict', 'optimizer', 'print_lr', 'state_dict', 'step', 'step_size', 'verbose']


In [91]:
# 학습
total_batch = len(train_loader)
for epoch in range(num_epoch):
  scheduler.step()
  avg_cost = 0
  #scheduler.step(loss.cpu().data.numpy()) # only for ReduceLRONPlateau
  for X, Y in train_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    output = model(X)
    cost = loss_func(output, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
  print('Epoch : {}, lr : {}, Cost : {}'.format(epoch + 1, scheduler.get_lr(), avg_cost))

Epoch : 1, lr : [0.000970299], Cost : 1.856903314590454


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:536: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  _warn_get_lr_called_within_step(self)


Epoch : 2, lr : [0.0009605960099999999], Cost : 0.3945898115634918


In [92]:
# 테스트
with torch.no_grad():
  X = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y = mnist_test.targets.to(device)

  output = model(X)
  prediction = torch.argmax(output, dim=1) == Y
  correct_prediction = prediction.float().sum()

print('정확도 : {}'.format(100 * correct_prediction / len(mnist_test)))

정확도 : 90.27999877929688


# Data Normalization
- Normalize MNIST Data

In [93]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [94]:
# 하이퍼파라미터 설정
batch_size = 16
learning_rate = 0.0002
num_epoch = 1

In [105]:
# 데이터 다운로드
# Normalize의 mean (R, G, B) 각 채널별로 평균 값을 주면 됨
# ToTensor()만 거치면 0~1사이의 값으로 바뀜
mnist_train = dsets.MNIST('./', train=True,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize(mean=(0.1307,), std=(0.3081,))
                          ]),
                          download=True)
mnist_test = dsets.MNIST('./', train=False,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize(mean=(0.1307,), std=(0.3081,))
                         ]),
                         download=True)

# Batch Normalization

In [106]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [107]:
batch_size = 16
learning_rate = 0.0002
num_epoch = 1

In [108]:
mnist_train = dsets.MNIST("./", train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST("./", train=False, transform=transforms.ToTensor(), download=True)

In [109]:
# 데이터로더
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)

In [115]:
# CNN 모델
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer = nn.Sequential(
        nn.Conv2d(1, 16, 3, padding=1), # 28
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.Conv2d(16, 32, 3, padding=1), # 28
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2, 2), # 14
        nn.Conv2d(32, 64, 3, padding=1), # 14
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2, 2) # 7
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(64 * 7 * 7, 100),
        nn.BatchNorm1d(100),
        nn.ReLU(),
        nn.Linear(100, 10)
    )

  def forward(self, x):
    out = self.layer(x)
    out = out.view(out.size(0), -1)
    out = self.fc_layer(out)
    return out

model = CNN().to(device)

In [111]:
# 손실 함수 및 optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [120]:
# Train
total_batch = len(train_loader)
for epoch in range(5):
  avg_cost = 0
  for X, Y in train_loader:
    X = X.to(device)
    Y = Y.to(device)
    optimizer.zero_grad()
    output = model(X)
    cost = criterion(output, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
  print('Epoch : {}, Cost : {}'.format(epoch + 1, avg_cost))

Epoch : 1, Cost : 2.3775434494018555
Epoch : 2, Cost : 2.3775386810302734
Epoch : 3, Cost : 2.377542734146118
Epoch : 4, Cost : 2.3775405883789062
Epoch : 5, Cost : 2.3775441646575928


In [130]:
# 테스트
model.eval() # 배치정규화 층 비활성화

with torch.no_grad():
  X = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y = mnist_test.targets.to(device)

  output = model(X)
  prediction = torch.argmax(output, dim=1) == Y
  correct_prediction = prediction.float().sum()

print('정확도 : {}'.format(100 * correct_prediction / len(mnist_test)))

정확도 : 8.880000114440918


# Gradient Descent Variants
- SGD
- Momentum
- Nestrov
- Adagrad
- Adadelta
- Adam

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [8]:
# 하이퍼파라미터 설정
batch_size = 16
learning_rate = 0.0002
num_epoch = 1

In [3]:
# 데이터
mnist_train = dsets.MNIST('./', train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST('./', train=False, transform=transforms.ToTensor(), download=True)

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 9.91M/9.91M [00:01<00:00, 5.18MB/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 28.9k/28.9k [00:00<00:00, 153kB/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.45MB/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 4.54k/4.54k [00:00<00:00, 7.76MB/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [4]:
# 데이터로더
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

In [5]:
# CNN 모델
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer = nn.Sequential(
        nn.Conv2d(1, 16, 3, padding=1), # 28
        nn.ReLU(),
        nn.Conv2d(16, 32, 3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2), # 14
        nn.Conv2d(32, 64, 3, padding=1), # 14
        nn.ReLU(),
        nn.MaxPool2d(2, 2) # 7
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(64 * 7 * 7, 100),
        nn.ReLU(),
        nn.Linear(100, 10)
    )

  def forward(self, x):
    out = self.layer(x)
    out = out.view(out.size(0), -1)
    out = self.fc_layer(out)

    return out

model = CNN()

In [6]:
# 손실 함수 및 optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# 학습
total_batch = len(train_loader)
for epoch in range(num_epoch):
  avg_cost = 0
  for X, Y in train_loader:

    optimizer.zero_grad()
    output = model(X)
    cost = criterion(output, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print('[Epoch {}] Cost : {}'.format(epoch + 1, avg_cost))

[Epoch 1] Cost : 0.044940751045942307


In [10]:
# 테스트
with torch.no_grad():
  X = mnist_test.data.view(len(mnist_test), 1, 28, 28).float()
  Y = mnist_test.targets

  output = model(X)
  prediction = torch.argmax(output, dim=1) == Y
  correct_prediction = prediction.float().sum()

print('정확도 : {}'.format(100 * correct_prediction / len(mnist_test)))

정확도 : 98.58000183105469
